# VLE calculation for fluid mixtures with SAFT-$\gamma$-Mie

In this notebook, VLE calculations for fluid mixtures with ```saftgammamie``` EoS are illustrated.

First, all the needed modules are imported.

- numpy: numerical interface and work with arrays
- matplotlib: to plot results
- sgtpy: package with SAFT-$\gamma$-Mie EoS and phase equilibria functions.

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from sgtpy import component, mixture, saftgammamie

Now, pure components are configured and created with the ```component``` function. Then, a mixture is created with them using the ```mixture``` function or by adding (`+`) pure components. The interaction parameters are set up with the ```mixture.saftgammamie``` method. Finally, the ```eos``` object is created with the ```saftgammamie``` function.

The ```eos``` object includes all the necessary methods to compute phase equilibria and interfacial properties using SAFT-$\gamma$-Mie EoS.

For this notebook, all calculations are exemplified for a mixture of ethanol and water.

In [17]:
ethanol = component(GC={'CH3':1, 'CH2OH':1})
water = component(GC={'H2O':1})
co2 = component(GC={'CO2': 1 })

# creating mixture by adding pure components
mix = ethanol + water + co2

mix.saftgammamie()
eos = saftgammamie(mix)

## Flash calculation

This type of equilibria calculation is carried out at a given global composition (z), temperature (K), and pressure (Pa) and can be used for vapor/liquid and liquid/liquid equilibria. 

The flash calculation is carried out with the ```flash``` function, and it requires initial guesses for the phase composition ```x0```, ```y0```, and their aggregation state (```LV``` for liquid/vapor equilibria).

The ```full_output``` option allows obtaining all the information related to the flash calculation. This includes the volume of the phases and the solution for the fraction of the nonbonded sites of each phase.

**new in v0.0.19:** 
- The `flash` now includes the `not_in_x_list` and `not_in_y_list` attributes, where the user can provide the indices of a component not present in a phase.
- The user can choose to minimize the Gibbs free energy using a Gibbs approach (T,P) or a Helmholtz approach (T,V). The minimization scheme is set the `minimization_approach` option to either `gibbs` or `helmholtz`.

In [18]:
from sgtpy.equilibrium import flash

# flash conditions
T = 350.  # K
P = 1e5  # Pa
z = np.array([0.3, 0.65, 0.05])
# initial guesses for liquid and vapor phase composition 
x0 = np.array([0.1, 0.9, 0.])
y0 = np.array([0.5, 0.1, 0.4])
flash(x0, y0, 'LV', z, T, P, eos, full_output=True)

      T: 350.0
      P: 100000.0
   beta: 0.2402074320827491
  error: 8.979956459572758e-09
   iter: 10
      X: array([2.55129874e-01, 7.44616924e-01, 2.53201700e-04])
     v1: 2.8526639184704464e-05
  Xass1: array([0.32687072, 0.1670568 , 0.08427193, 0.25425129, 0.40740223,
       1.        ])
 state1: 'L'
      Y: array([0.44192728, 0.35072018, 0.20735253])
     v2: 0.02830477102317112
  Xass2: array([0.98849193, 0.98219415, 0.9753774 , 0.99012372, 0.99851429,
       1.        ])
 state2: 'V'
 method: 'ASS'

Using the `not_in_x_list` and `not_in_y_list` options can be useful if one component is known to be non-condensable or non-volatile. For example, in the previous example, we can neglect the amount of CO2 in the liquid phase.

In [19]:
# flash conditions
T = 350.  # K
P = 1e5  # Pa
z = np.array([0.3, 0.65, 0.05])
# initial guesses for liquid and vapor phase composition 
x0 = np.array([0.1, 0.9, 0.])
y0 = np.array([0.5, 0.1, 0.4])
# ignoring component of index 2 (co2) in the x phase (liquid)
flash(x0, y0, 'LV', z, T, P, eos, full_output=True, not_in_x_list=[2])

      T: 350.0
      P: 100000.0
   beta: 0.24142625364834083
  error: 1.938822717361671e-09
   iter: 10
      X: array([0.25477356, 0.74522644, 0.        ])
     v1: 2.850633874678591e-05
  Xass1: array([0.32702714, 0.16691912, 0.08424755, 0.25402487, 0.40721168,
       1.        ])
 state1: 'L'
      Y: array([0.44210382, 0.3507936 , 0.20710258])
     v2: 0.02830435712077443
  Xass2: array([0.9884874 , 0.98218935, 0.9753692 , 0.99012214, 0.99851396,
       1.        ])
 state2: 'V'
 method: 'ASS'

Similarly, if we reduce the temperature we could enforce the water content in the vapor phase to be zero.

In [20]:
# flash conditions
T = 298.15  # K
P = 1e5  # Pa
z = np.array([0.3, 0.65, 0.05])
# initial guesses for liquid and vapor phase composition 
x0 = np.array([0.1, 0.9, 0.])
y0 = np.array([0.5, 0.0, 0.4])

# ignoring component of index 1 (water) in the y phase (vapor)
flash(x0, y0, 'LV', z, T, P, eos, full_output=True, not_in_y_list=[1])

      T: 298.15
      P: 100000.0
   beta: 0.04819273221635788
  error: 3.2244916611207655e-11
   iter: 5
      X: array([0.31321667, 0.68291136, 0.00387197])
     v1: 2.9524906734011233e-05
  Xass1: array([0.17419899, 0.13475789, 0.03721939, 0.24265411, 0.28344979,
       1.        ])
 state1: 'L'
      Y: array([0.0389706, 0.       , 0.9610294])
     v2: 0.024667918662056056
  Xass2: array([0.99652318, 0.99826159, 0.99522624, 0.9950175 , 1.        ,
       1.        ])
 state2: 'V'
 method: 'ASS'

The `flash` solver will attempt to obtain the phase equilibria using accelerated successive substitution (ASS) for `nacc` cycles. If the solver fails to reach a given tolerance it will attempt to minimize the Gibbs free energy. 

By default, the `flash` solver uses a Gibbs approach (T,P) (`minimization_approach='gibbs'`). (For demonstration purposes, `nacc=0` prevents using ASS. Hence, the solver will directly attempt to minimize the Gibbs free energy).

In [21]:
# flash conditions
T = 350.  # K
P = 1e5  # Pa
z = np.array([0.3, 0.65, 0.05])
# initial guesses for liquid and vapor phase composition 
x0 = np.array([0.25, 0.75, 0.])
y0 = np.array([0.4, 0.3, 0.3])

flash(x0, y0, 'LV', z, T, P, eos, full_output=True, nacc=0, minimization_approach='gibbs')

      T: 350.0
      P: 100000.0
   beta: 0.2402108544635122
  error: 1.428623252412305e-07
   iter: 7
      X: array([2.55127795e-01, 7.44619015e-01, 2.53189941e-04])
     v1: 2.8526339315620294e-05
  Xass1: array([0.3268755 , 0.16705438, 0.0842723 , 0.2542467 , 0.40740147,
       1.        ])
 state1: 'L'
      Y: array([0.4419312 , 0.35071918, 0.20734962])
     v2: 0.028304756264170724
  Xass2: array([0.98849159, 0.9821941 , 0.975377  , 0.99012382, 0.99851431,
       1.        ])
 state2: 'V'
 method: 'Gibbs_minimization'

Alternatively, the Gibbs free energy can be minimized in a Helmholtz approach (T,V). This is done by setting `minimization_approach='helmholtz'`.

In [22]:
# flash conditions
T = 350.  # K
P = 1e5  # Pa
z = np.array([0.3, 0.65, 0.05])
# initial guesses for liquid and vapor phase composition 
x0 = np.array([0.25, 0.75, 0.])
y0 = np.array([0.4, 0.3, 0.3])

flash(x0, y0, 'LV', z, T, P, eos, full_output=True, nacc=0, minimization_approach='helmholtz')

      T: 350.0
      P: 100000.0
   beta: 0.24020335197149673
  error: 2.778165707564724e-06
   iter: 21
      X: array([2.55130669e-01, 7.44616128e-01, 2.53203311e-04])
     v1: 2.852645585732904e-05
  Xass1: array([0.32687364, 0.16705532, 0.08427215, 0.25424849, 0.40740176,
       1.        ])
 state1: 'L'
      Y: array([0.44192794, 0.35071601, 0.20735604])
     v2: 0.028304767359167222
  Xass2: array([0.98849168, 0.98219425, 0.97537719, 0.99012389, 0.99851432,
       1.        ])
 state2: 'V'
 method: 'Helmholtz_minimization'

## Bubble point calculation

This type of equilibria calculation is carried out at a given composition (x) of a liquid phase and fixed temperature (K) or pressure (Pa).

The function ```bubblePy``` is used to compute equilibrium pressure and vapor phase composition and it requires initial guesses of these variables. On the other hand, the function ```bubbleTy``` is used to compute equilibrium temperature and vapor phase composition, this function requires initial guesses of those variables.

For both functions, the ```full_output``` option allows obtaining all the information related to the bubble point calculation. This includes the volume of the phases and the solution for the fraction of the nonbonded sites of each phase.

Similarly, as for flash calculation, initial guesses for the volume roots and nonbonded association sites can be supplied to the function using the ```v0``` and ```Xass0``` options.

**new in v0.0.19:** 
- The `bubblePy` and `bubbleTy` now include the `not_in_y_list` attribute, where the user can provide the indices of a component not present in the vapor phase.

In [23]:
# bubble point x, T -> y, P
from sgtpy.equilibrium import bubblePy

# bubble point conditions
T = 298.15  # K
x = np.array([0.3, 0.65, 0.05])

# initial guesses for pressure and vapor phase composition 
P0 = 1e6  # Pa
y0 = np.array([0.05, 0.05, 0.9])

bubblePy(y0, P0, x, T, eos, full_output=True)

      T: 298.15
      P: 1235106.8449412852
  error: 3.924193192617054e-09
   iter: 5
      X: array([0.3 , 0.65, 0.05])
     v1: 2.994560566699131e-05
  Xassl: array([0.18978019, 0.1312964 , 0.04199955, 0.22948589, 0.31152038,
       1.        ])
 state1: 'Liquid'
      Y: array([0.00317881, 0.00250471, 0.99431648])
     v2: 0.0018888540966911479
  Xassv: array([0.99613794, 0.99409462, 0.99224398, 0.9345382 , 0.99968386,
       1.        ])
 state2: 'Vapor'
 method: 'quasi-newton + ASS'

In [24]:
# bubble point x, P -> y, T
from sgtpy.equilibrium import bubbleTy

P = 1e6  # Pa
x = np.array([0.3, 0.65, 0.05])

# initial guesses for temperature and vapor phase composition 
T0 = 290.  # K
y0 = np.array([0.05, 0.05, 0.9])

bubbleTy(y0, T0, x, P, eos, full_output=True)

      T: 290.21776723496436
      P: 1000000.0
  error: 6.781908368248228e-12
   iter: 3
      X: array([0.3 , 0.65, 0.05])
     v1: 2.9736262625973365e-05
  Xassl: array([0.17083297, 0.12390204, 0.03627919, 0.22195373, 0.28936466,
       1.        ])
 state1: 'Liquid'
      Y: array([0.00233814, 0.00189083, 0.99577103])
     v2: 0.0022896104321662616
  Xassv: array([0.99712097, 0.99553511, 0.99420737, 0.93828479, 0.99977341,
       1.        ])
 state2: 'Vapor'
 method: 'quasi-newton + ASS'

Using the `not_in_y_list` option, the user can ignore a component in the vapor phase. Here, for example, we ignore the water content in the vapor phase.

In [25]:
# ignoring the water (index=1) content in the vapor phase
bubblePy(y0, P0, x, T, eos, full_output=True, not_in_y_list=[1])

      T: 298.15
      P: 1231770.3637951638
  error: 1.778674096897248e-09
   iter: 5
      X: array([0.3 , 0.65, 0.05])
     v1: 2.9945646067732962e-05
  Xassl: array([0.1897806 , 0.13129653, 0.04199966, 0.22948606, 0.31152098,
       1.        ])
 state1: 'Liquid'
      Y: array([0.00316077, 0.        , 0.99683923])
     v2: 0.0018948236933457104
  Xassv: array([0.99626892, 0.99813446, 0.99489504, 0.93703149, 1.        ,
       1.        ])
 state2: 'Vapor'
 method: 'quasi-newton + ASS'

In [26]:
# ignoring the water (index=1) content in the vapor phase
bubbleTy(y0, T0, x, P, eos, full_output=True, not_in_y_list=[1])

      T: 290.29070795801107
      P: 1000000.0
  error: 3.037003981677987e-11
   iter: 3
      X: array([0.3 , 0.65, 0.05])
     v1: 2.9738183166535964e-05
  Xassl: array([0.17100669, 0.12396841, 0.03632978, 0.2220217 , 0.28957063,
       1.        ])
 state1: 'Liquid'
      Y: array([0.00233481, 0.        , 0.99766519])
     v2: 0.0022907207599434276
  Xassv: array([0.99719955, 0.99859978, 0.99620664, 0.94015   , 1.        ,
       1.        ])
 state2: 'Vapor'
 method: 'quasi-newton + ASS'

## Dew point calculation

This type of equilibria calculation is carried out at a given composition (y) of a vapor phase and fixed temperature (K) or pressure (Pa).

The function ```dewPx``` is used to compute equilibrium pressure and liquid phase composition and it requires initial guesses of these variables. On the other hand, the function ```dewTx``` is used to compute equilibrium temperature and liquid phase composition; this function requires initial guesses of those variables.

For both functions, the ```full_output``` option allows obtaining all the information related to the dew point calculation. This includes the volume of the phases and the solution for the fraction of the nonbonded sites of each phase.

Similarly, as for flash calculation, initial guesses for the volume roots and nonbonded association sites can be supplied to the function using the ```v0``` and ```Xass0``` options.

**new in v0.0.19:** 
- The `dewTx` and `dewPx` now include the `not_in_x_list` attribute, where the user can provide the indices of a component not present in the liquid phase.

In [27]:
# dew point y, T -> x, P
from sgtpy.equilibrium import dewPx

T = 350.  # K
y = np.array([0.45, 0.35, 0.2])

# initial guesses for pressure and liquid phase composition 
P0 = 1e5  # Pa
x0 = np.array([0.5, 0.5, 0.0])

dewPx(x0, P0, y, T, eos, full_output=True)

      T: 350.0
      P: 99680.9296526336
  error: 2.7278179715040096e-13
   iter: 3
      X: array([2.64442372e-01, 7.35303958e-01, 2.53670111e-04])
     v1: 2.8903512836906392e-05
  Xassl: array([0.3209348 , 0.17011953, 0.08381217, 0.26005726, 0.40836279,
       1.        ])
 state1: 'Liquid'
      Y: array([0.45, 0.35, 0.2 ])
     v2: 0.028386749027522112
  Xassv: array([0.98833189, 0.98217628, 0.97518637, 0.99017924, 0.99852154,
       1.        ])
 state2: 'Vapor'
 method: 'quasi-newton + ASS'

In [28]:
# dew point y, P -> x, T
from sgtpy.equilibrium import dewTx

P = 1e5  # Pa
y = np.array([0.45, 0.35, 0.2])

# initial guesses for temperature and liquid phase composition 
T0 = 350.  # K
x0 = np.array([0.5, 0.5, 0.0])

dewTx(x0, T0, y, P, eos, full_output=True)

      T: 350.07786504963815
      P: 100000.0
  error: 2.3181456754175954e-13
   iter: 3
      X: array([2.64524435e-01, 7.35221242e-01, 2.54323907e-04])
     v1: 2.890905588156929e-05
  Xassl: array([0.32105138, 0.17022954, 0.08388805, 0.26018886, 0.40854495,
       1.        ])
 state1: 'Liquid'
      Y: array([0.45, 0.35, 0.2 ])
     v2: 0.028301036176470343
  Xassv: array([0.98831338, 0.98214815, 0.97514626, 0.99016251, 0.99851848,
       1.        ])
 state2: 'Vapor'
 method: 'quasi-newton + ASS'

Using the `not_in_x_list` option, the user can ignore a component in the liquid phase. Here, for example, we ignore the CO2 content in the liquid phase.

In [29]:
# ignoring the CO2 (index=2) content in the liquid phase
dewPx(x0, P0, y, T, eos, full_output=True, not_in_x_list=[2])

      T: 350.0
      P: 99707.35099265908
  error: 1.1927125953548057e-12
   iter: 3
      X: array([0.26399328, 0.73600672, 0.        ])
     v1: 2.8879389885556734e-05
  Xassl: array([0.32114415, 0.16994934, 0.08379204, 0.2597702 , 0.40815887,
       1.        ])
 state1: 'Liquid'
      Y: array([0.45, 0.35, 0.2 ])
     v2: 0.028379013203053274
  Xassv: array([0.9883288 , 0.9821716 , 0.97517986, 0.99017665, 0.99852114,
       1.        ])
 state2: 'Vapor'
 method: 'quasi-newton + ASS'

In [30]:
# ignoring the CO2 (index=2) content in the liquid phase
dewTx(x0, T0, y, P, eos, full_output=True, not_in_x_list=[2])

      T: 350.0713905915723
      P: 100000.0
  error: 7.283063041541292e-14
   iter: 3
      X: array([0.26406752, 0.73593248, 0.        ])
     v1: 2.8884413173493584e-05
  Xassl: array([0.32125137, 0.17004973, 0.08386153, 0.25989014, 0.40832529,
       1.        ])
 state1: 'Liquid'
      Y: array([0.45, 0.35, 0.2 ])
     v2: 0.028300418153765432
  Xassv: array([0.98831181, 0.9821458 , 0.97514307, 0.9901613 , 0.99851833,
       1.        ])
 state2: 'Vapor'
 method: 'quasi-newton + ASS'

For further information of any of these functions just run: ```function?```